<a href="https://colab.research.google.com/github/abishek/learning-anlp/blob/master/basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Understanding and implementing TF-IDF for a corpus.

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('gutenberg')
nltk.download('shakespeare')
from nltk.probability import FreqDist
from nltk.corpus import stopwords, shakespeare
import pandas as pd

# shakespeare corpus has a few plays in it. let me pick the first one.
play_to_use = shakespeare.fileids()[0]


stop_words = set(stopwords.words('english'))
#read the corpus
words = nltk.Text(nltk.corpus.shakespeare.words(play_to_use))
#convert to small letters
words=[word.lower() for word in words if word.isalpha() ]
words=[word.lower() for word in words if word not in stop_words ]
fDist = FreqDist(words)

heading = ['Word','Frequency']
tf_list = []
for x,v in fDist.most_common(10):
    tf_list.append((x,v))
print(pd.DataFrame(tf_list,columns=heading))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package shakespeare to /root/nltk_data...
[nltk_data]   Package shakespeare is already up-to-date!
        Word  Frequency
0     antony        387
1     caesar        290
2  cleopatra        274
3       mark        264
4       thou        182
5  enobarbus        152
6   domitius        140
7   octavius        133
8      shall        127
9      enter        112
